In [ ]:
# ------------------------------------------------------
# IMPORT DES LIBRAIRIES
# ------------------------------------------------------
import pandas as pd               # pour lire et manipuler des données tabulaires (CSV)
import numpy as np                # pour manipuler des tableaux numériques
from sklearn.model_selection import train_test_split, StratifiedKFold  # pour split train/test et validation croisée
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler  # normalisation des données
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score  # métriques pour évaluer les modèles
from xgboost import XGBClassifier  # modèle XGBoost (boosting gradient)
from imblearn.over_sampling import SMOTE  # pour rééquilibrer les classes rares
import warnings
warnings.filterwarnings("ignore")  # éviter les warnings qui alourdissent la console

# ------------------------------------------------------
# CONFIGURATION GENERALE
# ------------------------------------------------------
FEATURES_PATH = "data/features.csv"   # chemin vers le CSV contenant les données
TARGET_COLUMN = "label"               # nom de la colonne cible (à prédire)
SEED = 42                             # graine aléatoire pour que les résultats soient reproductibles

# ------------------------------------------------------
# CHARGEMENT DU DATASET
# ------------------------------------------------------
print("📥 Chargement du dataset...")
df = pd.read_csv(FEATURES_PATH)  # lecture du CSV dans un DataFrame

# Séparation des features (X) et de la cible (y)
X = df.drop(columns=[TARGET_COLUMN]).values  # toutes les colonnes sauf la cible
y = df[TARGET_COLUMN].values                 # colonne cible

print(f"✅ Dataset chargé : {X.shape[0]} échantillons et {X.shape[1]} features.")

# ------------------------------------------------------
# SCALERS
# ------------------------------------------------------
# Les scalers transforment les données pour qu’elles soient sur la même échelle
scalers = {
    "StandardScaler": StandardScaler(),  # moyenne = 0, variance = 1
    "MinMaxScaler": MinMaxScaler(),      # min = 0, max = 1
    "RobustScaler": RobustScaler()       # moins sensible aux outliers
}

# ------------------------------------------------------
# CONFIGURATIONS XGBOOST
# ------------------------------------------------------
configs = {
    "Baseline": {"n_estimators": 200, "learning_rate": 0.1, "max_depth": 6},  # modèle standard
    "Deep Trees": {"n_estimators": 300, "learning_rate": 0.05, "max_depth": 10},  # arbres profonds
    "Shallow Trees": {"n_estimators": 500, "learning_rate": 0.01, "max_depth": 3}  # arbres peu profonds
}

# ------------------------------------------------------
# FONCTION D'EVALUATION
# ------------------------------------------------------
def evaluate(y_true, y_pred, dataset_name="Test"):
    """
    Calcule et affiche :
    - Accuracy : proportion de bonnes prédictions
    - F1-score : équilibre entre précision et rappel, pondéré pour multi-classes
    - ROC-AUC : performance du modèle pour chaque classe (si binaire ou multi-classes en one-hot)
    """
    acc = accuracy_score(y_true, y_pred)  # proportion de bonnes prédictions
    f1 = f1_score(y_true, y_pred, average="weighted")  # F1-score pondéré pour multi-classes

    # ROC-AUC : utile pour voir la qualité de séparation des classes
    try:
        auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred), average="weighted")
    except:
        auc = None  # si multi-classes et non applicable

    # Affichage des métriques
    print(f"📊 {dataset_name} | Accuracy = {acc:.4f}, F1 = {f1:.4f}", end="")
    if auc is not None:
        print(f", ROC-AUC = {auc:.4f}")
    else:
        print(" (ROC-AUC non calculable - multi-classes détecté)")

    return acc, f1, auc

# ------------------------------------------------------
# BOUCLE PRINCIPALE
# ------------------------------------------------------
results = []  # liste pour stocker les résultats

# 1️⃣ Boucle sur chaque scaler
for scaler_name, scaler in scalers.items():
    # Normalisation des données pour que toutes les features soient comparables
    X_scaled = scaler.fit_transform(X)

    # Split TRAIN/TEST (80%/20%) avec stratification pour conserver la proportion des classes
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, stratify=y, random_state=SEED
    )

    # 2️⃣ Boucle sur chaque configuration XGBoost
    for config_name, params in configs.items():
        print(f"\n🚀 Test avec {scaler_name} + Config = {config_name}")

        # Création du modèle XGBoost avec paramètres choisis
        model = XGBClassifier(
            use_label_encoder=False,  # pour éviter warnings
            eval_metric="logloss",    # métrique interne pour XGBoost
            random_state=SEED,
            **params
        )

        # ------------------------------------------------------
        # 3️⃣ Validation croisée (Stratified K-Fold)
        # ------------------------------------------------------
        # Sépare les données d'entraînement en K folds pour évaluer le modèle de manière robuste
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        f1_scores = []  # stocke les F1-score pour chaque fold

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
            # Séparer le fold courant en TRAIN et VALIDATION
            X_tr, X_val = X_train[train_idx], X_train[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            # ------------------------------------------------------
            # 4️⃣ Rééquilibrage des classes avec SMOTE
            # ------------------------------------------------------
            smote = SMOTE(random_state=SEED)          # objet SMOTE
            X_tr_bal, y_tr_bal = smote.fit_resample(X_tr, y_tr)  # création de nouvelles instances pour classes rares

            # Entraînement du modèle sur données équilibrées
            model.fit(X_tr_bal, y_tr_bal)

            # Prédictions sur le fold de validation
            y_val_pred = model.predict(X_val)

            # Calcul du F1-score pour ce fold
            f1_fold = f1_score(y_val, y_val_pred, average="weighted")
            f1_scores.append(f1_fold)

            print(f"   Fold {fold+1} : F1 = {f1_fold:.4f}")

        # Moyenne et écart type des scores F1 sur tous les folds
        print(f"   ➡️ Moyenne F1 CV = {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")

        # ------------------------------------------------------
        # 5️⃣ Évaluation finale sur le TEST set
        # ------------------------------------------------------
        smote = SMOTE(random_state=SEED)
        X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)  # rééquilibrage sur tout le TRAIN
        model.fit(X_train_bal, y_train_bal)  # réentraînement sur tout le TRAIN

        y_test_pred = model.predict(X_test)  # prédiction sur le TEST set

        # Évaluation des métriques sur le TEST set
        acc, f1, auc = evaluate(y_test, y_test_pred, dataset_name="Test final")

        # Stockage des résultats pour comparatif
        results.append({
            "Scaler": scaler_name,
            "Config": config_name,
            "CV_F1_mean": np.mean(f1_scores),
            "CV_F1_std": np.std(f1_scores),
            "Test_Accuracy": acc,
            "Test_F1": f1,
            "Test_AUC": auc
        })

# ------------------------------------------------------
# AFFICHAGE DES RESULTATS
# ------------------------------------------------------
results_df = pd.DataFrame(results)
print("\n📊 Tableau comparatif des résultats (toutes variantes) :")
print(results_df)
